# Imports used for recommendation using Scikit-Surprise library

In [19]:
import os
import csv
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict

# Specifying the mongoDB root directory and exporting data using mongoexport as csv file

In [11]:
# File Path where the files for train ,test and output needs to be stored.
pr_file_path="/Users/ashokkumar/Desktop/ApacheTomcat/apache-tomcat-7.0.34/webapps/BestDealWebsite"

# Location of the mongoexport that is used to ecport data from mondo DB
os.chdir('/Users/ashokkumar/Desktop/MongoDB/mongodb-osx-x86_64-3.2.22/bin')
os.system('./mongoexport --db CustomerReviews --collection myReviews --type csv --fields userName,productName,reviewRating --out '+pr_file_path+'/mongodata_train.csv')


0

In [5]:
import pymysql
import pandas

In [16]:
import pymysql
import pandas

conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='root', database='SocialRecommendation')
cursor = conn.cursor()
query = 'select userid, name, reviewrating  from review'

In [17]:
results = pandas.read_sql_query(query, conn)
results.to_csv("data_train.csv", index=False)

# Iterating over the exported file from mongo DB to generate test file.

In [20]:

# Loop over the trainb file exported from mongo DB to generate test file.
with open("data_train.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open("data_test.csv", "w",newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)
            
file_path = os.path.expanduser('data_test.csv')

# Recommendation logic using surprise library and predicting top 3 results

In [23]:
# As we're loading a custom dataset, we need to define a reader. 
reader = Reader(line_format='user item rating', sep=',')

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    
out = open('output.csv', 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


user1 ['Patel Brothers', 'Jewel-Osco', 'Quigley', 'Bar Louie - Naperville', 'Pita Inn', 'Skokie Fashion Square', 'Lickity Split Frozen Custard', 'Shedd Aquarium', 'Parivar Grocers', 'Pub 56']
user2 ['Millennium Park', 'Patel Brothers', 'Skokie Fashion Square', 'Quigley', 'Bombay Wraps', 'Captains Walk Shopping Mall', 'India House Restaurant - Chicago', 'Pita Inn', 'Parivar Grocers', 'Bar Louie - Naperville']
user4 ['Pita Inn', 'India House Restaurant - Chicago', 'Bombay Wraps', 'Lickity Split Frozen Custard', 'Millennium Park', 'Skokie Fashion Square', 'Quigley', 'Shedd Aquarium', 'Taste of Peru', 'Bar Louie - Naperville']
user5 ['Parivar Grocers', 'Pita Inn', 'Taste of Peru', 'Patel Brothers', 'Millennium Park', 'India House Restaurant - Chicago', 'Skokie Fashion Square', 'Lickity Split Frozen Custard', 'Bombay Wraps', 'Jewel-Osco']
user3 ['Patel Brothers', 'Shedd Aquarium', 'Quigley', 'India House Restaurant - Chicago', 'Millennium Park', 'Bombay Wraps', 'Taste of Peru', 'Jewel-Osco'